In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
    from DCS import *
    from sentences import *
    import os
    from utilities import *
    import re
    from romtoslp import rom_slp
    import pickle

In [3]:
dcsPath = '../Text Segmentation/DCS_pick/'
dcsList = os.listdir(dcsPath)

In [4]:
sktPath_1 = '../TextSegmentation/Pickle_Files//'
sktList_1 = os.listdir(sktPath_1)

sktPath_2 = '../TextSegmentation/corrected_10to20/'
sktList_2 = os.listdir(sktPath_2)

In [17]:
preList = pickle.load(open('pvb.p', 'rb'))
def removePrefix(preList, lemma):
    for pre in preList:
        m = re.match(pre, lemma)
        if(m != None):
            s = m.span()
            pat = lemma[s[0]:s[1]]
            return (lemma.split(pat)[1])
removePrefix(preList, 'ASraya')

'Sraya'

In [5]:
def pickleFixLoad(filename):
    return pickle.load(open(filename, 'rb'), encoding=u'utf8')

In [6]:
def CanIUseIt(sntcObj, dcsObj):
#     print('-'*15)
#     CHECK 1 -> NUMBER OF CHUNKS/LEMMAS
    s = re.findall("[^ ]+", dcsObj.sentence)
    if((len(s) != len(dcsObj.lemmas)) or (len(s) != len(sntcObj.chunk))):
        return 1
    
#     CHECK 2 -> ALL LEMMAS PRESENT IN CORRESPONDING CHUNK
    sntc_lemma_packs = []
    for chunk in sntcObj.chunk:
        allwords = set()
        for pos, word_senses in chunk.chunk_words.items():
            for sense in word_senses:
                for lemma in sense.lemmas:
                    term = rom_slp(lemma.split('_')[0])
                    allwords.add(term)
#                     allwords.update([term, removePrefix(preList, term)])
                    
        sntc_lemma_packs.append(allwords)
        
    i = -1
    for chunk in dcsObj.lemmas:
        chunk = list(map(rom_slp, chunk))
        i += 1
        print(" | ".join(chunk))
        print("vs")
        print(sntc_lemma_packs[i])
        print()
        for lemma in chunk:
            if(lemma not in sntc_lemma_packs[i]):
                if lemma[removePrefix(preList, lemma)::] not in sntc_lemma_packs[i]:
#                     print(dcsObj.sentence)
                    return 2
#                     print('HUH')
    return 0
    
#     CHECK 3 -> NO CNG ERRORS ('VERBFORMS')


# common = []
# bads = []
# count = 0
# for dcsFile in dcsList[0:10000]:
#     if(validatePickleName(dcsFile) != ""):
#         if dcsFile in sktList_1:
#             count += 1
#             if(count % 100 == 0):
#                 print('Checkpoint: ', len(common), len(bads))                
#                 pass
#             common.append(dcsFile)
#             try:
#                 dcsObj = pickleFixLoad(dcsPath + dcsFile)
#                 sntcObj = pickleFixLoad(sktPath_1 + dcsFile)
#             except EOFError:
#                 continue
#             e = CanIUseIt(sntcObj, dcsObj)
#             # IF zero returned then all test passed
#             if e != 0:
#                 bads.append(dcsFile)
#             else:
# #                 print(dcsFile)
#                 pass
#             if e == 2:
# #                 print(dcsFile)
# #                 print("FAIL 2")
#                 pass
                

# good = set(common) - set(bads)
# print(len(good))

In [18]:
len(good)

NameError: name 'good' is not defined

In [19]:
len(bads)

NameError: name 'bads' is not defined

In [8]:
dcsFile = '434538.p'
dcsObj = pickleFixLoad(dcsPath + dcsFile)
sntcObj = pickleFixLoad(sktPath_2 + dcsFile)
e = CanIUseIt(sntcObj, dcsObj)
e

mamatva | aBiBU
vs
{'mamatva', 'vA', 'tu', 'Uta', 'enA', 'yuzmad', 'nAB', 'uta', 'BUta', 'aB', 'BU', 'Bu', 'na', 'asmad', 'mA'}

tad
vs
{'tad'}

tatra | eva
vs
{'', 'tatra', 'eva'}

parivft
vs
{'vft', 'par'}



0

In [3]:
# pickle.dump(good, open('goodSet_1.p', 'wb'))



mainset = set()
for file in os.listdir('good'):
    if '.p' in file:
        mainset = mainset.union(pickle.load(open('good/' + file, 'rb')))

len(mainset)

11739

In [7]:
goodDict = {}
for f in mainset:
    if(f in sktList_1):
        goodDict[f] = 'PF'
    elif f in sktList_2:
        goodDict[f] = 'C1020'

pickle.dump(goodDict, open('mergedGood_v3.p', 'wb'))
# goodDict = pickle.load(open('mergedGood_v3.p', 'rb'))

3